In [764]:
import numpy as np;
from tqdm import trange;
import pandas as pd;
from matplotlib import pyplot as plt;

In [765]:
print( "NumPy__version:", np.__version__ );
print( "Pandas__version:", pd.__version__ );

NumPy__version: 1.26.4
Pandas__version: 2.1.4


In [766]:
def xor_data():
    input_array = np.array( [ [ 0.0, 0.0 ], [ 0.0, 1.0 ], [ 1.0, 0.0 ], [ 1.0, 1.0 ] ] );
    output_array = np.array( [ [ 0.0 ], [ 1.0 ], [ 1.0 ], [ 0.0 ] ] );
    return ( input_array.astype('float32'), output_array.astype('float32') ),\
( input_array.astype('float32'), output_array.astype('float32') );

def load_data( name ):
    if name == "xor":
        return xor_data();

In [767]:
print( type( load_data( "xor" )[0][0] ) );

<class 'numpy.ndarray'>


In [768]:
class Loss( object ):
    def __init__( self, name : str ):
        self.name = name;

    def __call__( self, y_pred, y_true, deriv=False ):
        if self.name == "mse" and deriv==False:
            return np.mean( ( y_pred - y_true ) ** 2 );
    
        elif self.name == "cross_entropy" and deriv==False:
            return -np.mean( y_true * np.log( y_pred ) + ( 1 - y_true ) * np.log( 1 - y_pred ) );

        elif self.name == "cross_entropy" and deriv==True:
            # Clip predictions to avoid division by zero
            y_pred = np.clip( y_pred, 1e-15, 1 - 1e-15)

            # Compute the derivative of the binary cross-entropy loss
            derivative = (y_pred - y_true) / (y_pred * (1 - y_pred));
            return derivative
        
        else:
            raise ValueError( "Invalid loss function" );

In [769]:
def one_hot( Y ):
    """
    return an 0 vector with 1 only in the position correspondind to the value in Y
    """
    one_hot_Y = np.zeros( ( Y.max()+1, Y.size ) ) ;
    one_hot_Y[Y,np.arange( Y.size )] = 1; 
    return one_hot_Y;

class BackProp( object ):
    def __init__( self ) -> None:
        pass;

    def __call__( self, data : np.ndarray , labels : np.ndarray , size : int, logits : dict , trainable_variables : dict, transfer_function : callable ) -> dict:
        """
        """
        # compute gradients
        grads = {};
    
        loss_value = np.dot( 2 , logits['output'] - labels );
        dw2 = np.dot( 1/size, loss_value.dot( logits['hidden'].T ) );
        db2 = 1/size * np.sum( loss_value, 1 );

        dz1 = trainable_variables['W_hidden_output_layer'].T.dot( loss_value ) * transfer_function( logits['W_input_hidden_layer'], deriv=True );

        dw1 = 1/size * dz1.dot( data.T );
        db1 = 1/size * np.sum( dz1, 1 );

        grads['W_hidden_output_layer'] = dw2;
        grads['output'] = db2;
        grads['W_input_hidden_layer'] = dw1;
        grads['hidden'] = db1;

        error = np.sum( loss_value ** 2 ) / size;

        return grads, error;

class GradientDescent(object):
    """
    Gradient Descent optimizer.
    """
    def __init__(self, parameters: dict):
        self.parameters = parameters;
        self.name = "Gradient Descent";
    
    def minimize( self, trainable_variables, grads ):
        """
        """
        for key in trainable_variables.keys():
            if key[0] != 'W':
                trainable_variables[key] -= self.parameters['lr'] * np.reshape( grads[key], ( self.parameters['hidden_units'],1) );
            else:
                x = self.parameters['lr'] * grads[key];
                trainable_variables[key] -= x.T;

        return trainable_variables

In [770]:
class BaseLayer( object ):
    """
    Base class for the Layer class.
    """
    def __init__( self, hyperparams : dict, name : str  ):
        """
        Constructor for the Base Layer class.

        args:
            params : dict
            node_no : int

        returns:
            None

        attributes:
            self.params : dict
            self.node_no : int

        raises:
            None

        """
        self.hyperparams = hyperparams;
        self.name = name;

    def __rshift__( self, other ):
        """
        Overwrite the right shift operator to connect the layers.

        args:
            other : object

        returns:
            None

        attributes:
            None

        raises:
            None
        """
        return NeuralNetwork( self, other );

In [771]:
class InputLayer( BaseLayer ):
    """
    Input layer for the perceptron.
    """
    def __init__( self, hyperparams : dict, name: str  ):
        """
        Constructor for the Input Layer class.

        args:
            params : dict
            node_no : int

        returns:
            None

        attributes:
            self.bias : array

        raises:
            None
        """
        super( InputLayer, self ).__init__( hyperparams, name );
        self.node_no = hyperparams['input_units'];
    
    def output( self, inputs ):
        """
        Mirror the inputs.

        args:
            inputs : array

        returns:
            array

        attributes:
            None

        raises:
            None
        """
        self.bias_add = inputs;
        self.activated = inputs;

In [772]:
class Layer( BaseLayer ):
    """
    Layer class for the perceptron.
    """
    def __init__( self, hyperparams : dict, name: str, transfer: str  ):
        """
        Constructor for the Layer class.

        args:
            name : str
            params : dict
            node_no : int
            transfer : str

        returns:
            None

        attributes:
            self.name : str
            self.transfer : str
            self.inputs : object
            self.outputs : object

        raises:
            None
        """
        super( Layer, self ).__init__( hyperparams, name );
        self.transfer_name = transfer;

        self.inputs = None;
        self.outputs = None;
    
        if "hidden" in name:
            self.node_no = hyperparams['hidden_units'];
        else:
            self.node_no = hyperparams['output_units'];
        
        #create self.bias to be an array of size node_no with random values from a normal distribution between -1 and 1
        self.bias = np.random.rand( hyperparams['hidden_units'], hyperparams['examples'] ) - 0.5;
        self.bias = self.bias.astype('float32');

    def transfer_fx( self, inputs, deriv=False ):
        """
        Transfer function for the layer.

        args:
            inputs : array

        returns:
            array

        attributes:
            None

        raises:
            None
        """
        if self.transfer_name == "relu" and deriv == True:
            return inputs > 0;
    
        # implement the relu transfer function
        if self.transfer_name == "relu" and deriv == False:
            return np.maximum( 0,inputs );
        
        if self.transfer_name == 'softmax' and deriv == False:
            exp = np.exp( inputs - np.max( inputs ) );
            return exp / exp.sum( axis=0 );
            
        if self.transfer_name == "sigmoid" and deriv == True:
            print( "inputs:",inputs );
            print( "1-inputs:",1 - inputs );
            f_x = 1 / ( 1 + np.exp( -inputs ) );
            return f_x * ( 1 - f_x ); 
    
        # implement the sigmoid transfer function
        if self.transfer_name == "sigmoid" and deriv == False:
            print( "-inputs:",-inputs );
            print( "np.exp( -inputs )",np.exp( -inputs  ) );
            return 1 / ( 1 + np.exp( -inputs ) );


    def output( self, inputs, deriv=False ):
        """
        Calculate the output with the activation function and inputs.

        args:
            inputs : array

        returns:
            array

        attributes:
            self.outputs : array

        raises:
            None
        """
        self.bias_add = inputs + self.bias;
        if deriv == True:
            self.activated = self.transfer_fx( self.bias_add, deriv=True );
        else:
            self.activated = self.transfer_fx( self.bias_add );

In [773]:
class WeightLayer( BaseLayer ):
    """
    Weight layer for the perceptron.
    """
    
    def __init__( self, src : Layer, dest : Layer ) -> None:
        """
        Constructor for the weight layer.

        args:
            params : dict
            src : Layer
            dest : Layer

        returns:
            None

        attributes:
            self.src : Layer
            self.dest : Layer
            self.input_size : tuple
            self.output_size : tuple
            self.name : str
            self.weights : array

        raises: 
            None
        """
        self.src = src;
        self.dest = dest;
    
        self.weights = np.random.rand( self.src.node_no, self.dest.bias.shape[0] ) - 0.5;
        self.weights = self.weights.astype('float32');

        self.name    = "W_%s_%s_layer" % ( self.src.name, self.dest.name );

        self.src.outputs = self;
        self.dest.inputs = self;


    def output( self, inputs ):
        """
        Matrix multiplication between the inputs and the weights.

        args:
            inputs : array

        returns:
            array

        attributes:
            None

        raises:
            None
        """
        self.src.output( inputs );
        return self.src.bias_add, self.src.activated, self.weights.T.dot( self.src.activated );

In [774]:
class NeuralNetwork( object ):
    """
    This class respresents a Neural Networks.
    """
    
    def __init__( self, layer0, layer1 ):
        """
        Constructor for the Neural Network class.
        Creates a network with an input layer, layer0 and an output layer, layer1.

        args:
            layer0 : object
            layer1 : object

        returns:
            None

        attributes:
            self.hyperparams          : ( dict ) hyperparameters for the network.
            self.layers               : ( list ) list of layers in the network.
            self.layer_name           : ( list ) list of names of the layers in the network.
            self.input_layer          : ( object ) input layer of the network.
            self.output_layer         : ( object ) output layer of the network.
            self.weights              : ( list ) list of weights in the network.
            self.weight_names         : ( list ) list of names of the weights in the network.
            self.loss_fn              : ( object ) loss function for the network.
            self.optimizer            : ( object ) optimizer for the network.
            self.learnable_parameters : ( dict ) dictionary of learnable parameters in the network.

        raises:
            None 
        """
        # hyperparameters dictionary
        self.hyperparams = layer0.hyperparams;

        # layers
        self.layers = [ layer0, layer1 ];
        self.layer_name = [ layer0.name, layer1.name ];
        self.input_layer = layer0;
        self.output_layer = self.layers[-1];

        # weights
        self.weights = [ WeightLayer( layer0, layer1 ) ];
        self.weight_names = [ self.weights[0].name ];

        # loss function
        self.loss_fn = None;

        # optimizer
        self.optimizer = None;

        # initialize and populate learnable parameters used for learning
        self.trainable_parameters = {};
        for weight in self.weights:
            self.trainable_parameters[weight.name] = weight.weights;
        for layer in self.layers[1:]:
            self.trainable_parameters[layer.name] = layer.bias;

    
    def summary( self ):
        """
        Print the summary of the network.

        args:
            None

        returns:
            None

        attributes:
            None

        raises:
            None
        """
        print( "-------" );
        print( "| Summary |" );
        print( "-------" );
        print( f"Input Layer: { self.input_layer.node_no }" );
        print( f"Hidden Layer: { self.layers[1].node_no }" );
        print( f"Output Layer: { self.output_layer.node_no }" );
    
        print( "-------" );
        print( "| Weights |" );
        print( "-------" );
        for weight_obj in self.weights:
            print( f"{weight_obj.name}: \n {weight_obj.weights} , {weight_obj.weights.shape}" );
        
        print( "-------" );
        print( "| Bias |" );
        print( "------" );
        for layer_obj in self.layers:
            if hasattr( layer_obj, 'bias' ):
                print( f"{layer_obj.name}: \n {layer_obj.bias}, {layer_obj.bias.shape}" );
            else:
                print( f"{layer_obj.name}: " );
        
        print( "---------------" );
        print( "| Hyperparameters |" );
        print( "---------------" );
        print( f"Epochs: {self.hyperparams['epochs']}" );
        print( f"Learning Rate: {self.hyperparams['lr']}" );
        print( f"Minibatch Size: {self.hyperparams['minibatch_size']}" );
        print( "---------------" );

        print( "---------------" );
        print( f"| Loss Function | : {self.loss_fn.name}" );
        print( "---------------" );
        
        print( "---------------" );
        #print( f"| Optimizer | : {self.optimizer.name}" );
        print( "---------------" );

        print( "---------------" );
        print( "| Trainable Parameters | ")
        print( "---------------" );
        print( self.trainable_parameters );
    

    def __rshift__( self, other ):
        """
        Overwrite the right shift operator to add a layer to the network
        """
        if isinstance( other, Layer ) or isinstance( other, InputLayer ):
            # add weights between the layers, append the weight name to the list
            self.weights.append( WeightLayer( self.layers[-1], other ) );
            self.weight_names.append( self.weights[-1].name );
            
            # add the layer to the network, set the output layer to the last layer, and append the layer name to the list
            self.layers.append( other );
            self.output_layer = self.layers[-1];
            self.layer_name.append( other.name );

            # add the weights and biases to the trainable parameters
            self.trainable_parameters[self.weights[-1].name] = self.weights[-1].weights;
            self.trainable_parameters[other.name] = other.bias;

            return self;
    
        if isinstance( other, Loss ):
            self.loss_fn = other;
            return self;

        if isinstance( other, GradientDescent ):
            self.optimizer = other;
            return self;

        else:
            print( type( other ) );

    def output( self, inputs ):
        """
        Calculate the output of the network.

        args:
            inputs : array

        returns:
            array

        attributes:
            None

        raises:
            None
        """
        return self.output_layer.output( inputs );

    def forward( self, inputs ):
        """
        Forward pass through the network.

        args:
            inputs : array

        returns:
            None

        attributes:
            None

        raises:
            None
        """
        fwd_output = {};

        for i in range( len( self.weights ) ):
            src_bias_add, src_activated, inputs = self.weights[i].output( inputs );
            
            fwd_output[self.weights[i].name] = inputs;
            if self.layers[i].name == 'input':
                continue;
            fwd_output[self.layers[i].name] = src_activated;
            fwd_output[self.layers[i].name + "_bias_add"] = src_bias_add;
        
        self.output( inputs );
        fwd_output['output'] = self.output_layer.activated;
        fwd_output['output_bias_add'] = self.output_layer.bias_add;
        
        return fwd_output;

    def predict( self, X ):
        """
        Predict the output of the network.

        args:
            X : array

        returns:
            array

        attributes:
            None

        raises:
            None
        """
        logits = self.forward( X );
        return np.argmax( logits['output'], 0 );

    def get_accuracy( self, predictions, Y ):
        return np.sum( predictions == Y )/Y.size

    def train( self, X, y ):
        """
        Train the network.

        args:
            X : array
            y : array
            epochs : int
            lr : float

        returns:
            None

        attributes:
            None

        raises:
            None
        """

        """
        self.losses_pd = pd.DataFrame( columns = ['fwd_losses'], index=[0] );
        self.weights_pd = pd.DataFrame();
        self.bias_pd = pd.DataFrame();
        self.weight_layer_output_pd = pd.DataFrame();
        self.activated_pd  = pd.DataFrame();
        self.grad_pd = pd.DataFrame();

        for i in range( len( self.weights ) ):
            weights = self.weights[i];
            for l in range ( weights.weights.shape[0] ):
                for j in range( weights.weights.shape[1] ):
                    self.weights_pd.loc[ 0 ,  weights.name + f"{l}_{j}" ] = weights.weights[l][j];
        
        for i in range( len( self.layers ) ):
            layer = self.layers[i];
            if hasattr( layer, 'bias' ):
                for j in range( layer.bias.shape[0] ):
                    self.bias_pd.loc[ 0, layer.name + f"{j}" ] = layer.bias[j];
        
        
        #print( self.weights_pd )
        #print( self.bias_pd )
        """

        for epoch in range( self.hyperparams['epochs'] ):
  
            # forward pass
            #z1, a1, z2, a2 = self.forward( X );
            fwd_outputs = self.forward( X );

            #for item in fwd_outputs:
                #print( item, fwd_outputs[item].shape );

            # logging
            #for name in self.weight_names:
                #for i in range( fwd_outputs[name].shape[0] ):
                    #for j in range( fwd_outputs[name].shape[1] ):
                        #self.weight_layer_output_pd.loc[ 0, name + f"{i}_{j}" ] = fwd_outputs[name][i][j];
                
            #for name in self.layer_name[1:]:
                #for i in range( fwd_outputs[name].shape[0] ):
                    #for j in range( fwd_outputs[name].shape[1] ):
                       # self.activated_pd.loc[ 0, name + f"{i}_{j}" ] = fwd_outputs[name][i][j];
        

            # backward pass
            #one_hot_labels = one_hot( y )
            size = y.size;
            
            grads, error = BackProp()( X, y, size, fwd_outputs, self.trainable_parameters, self.layers[1].transfer_fx );
            
            # update the weights
            self.trainable_parameters = self.optimizer.minimize( self.trainable_parameters, grads );

            # update the weights and biases
            for weight in self.weights:
                weight.weights = self.trainable_parameters[weight.name];

            for layer in self.layers[1:]:
                layer.bias = self.trainable_parameters[layer.name];

            print( f"Epoch: {epoch+1} / {self.hyperparams['epochs']}, Error: {error}" );
            #if ( epoch+1 ) % int( self.hyperparams['epochs']/10 ) == 0:
                #print(f"Iteration: {epoch+1} / {self.hyperparams['epochs']}")
                #prediction = self.predict( X );
                #print(f'{self.get_accuracy(prediction, y):.3%}')


In [775]:
training, testing = load_data( "xor" );
x_train, y_train = training[0].T, training[1].T;
x_test, y_test = testing[0].T, testing[1].T;

In [776]:
params = {
    #seed
    "seed" : 42,

    "examples" : x_train.shape[1],
    
    # model hyperparameters
    "input_units" : x_train.shape[0],
    "hidden_units" : 2,
    "output_units" : 1,

    # optimizer hyperparameters
    "lr" : 0.15,

    # training hyperparameters
    "epochs" : 100,
    "minibatch_size" : 4
}
#np.random.seed(params['seed']);

In [777]:
input_layer = InputLayer( params, "input" );
hidden_layer = Layer( params, "hidden", "sigmoid" );
output_layer = Layer( params, "output", "sigmoid" );

loss = Loss( "cross_entropy" );
optimizer = GradientDescent( params );

In [778]:
nn = input_layer >> hidden_layer >> output_layer >> loss >> optimizer;


In [779]:
nn.train( x_train, y_train );

-inputs: [[ 0.3794611   0.18691097 -0.05070847 -0.7901538 ]
 [ 0.24823572 -0.02858008  0.65091646  0.3117391 ]]
np.exp( -inputs ) [[1.4614968  1.20552    0.9505558  0.45377502]
 [1.281762   0.9718244  1.9172971  1.3657982 ]]
-inputs: [[ 0.41937587 -0.21674185  0.4157359   0.05131501]
 [ 0.46019453 -0.41183862  0.00872019 -0.28185147]]
np.exp( -inputs ) [[1.521012   0.8051378  1.5154856  1.0526545 ]
 [1.5843822  0.66243124 1.0087583  0.75438565]]
inputs: [[ 0.         -0.06299692  0.40641746  0.34342054]
 [ 0.         -0.04600008 -0.39606413 -0.44206423]]
1-inputs: [[1.        1.0629969 0.5935825 0.6565795]
 [1.        1.0460001 1.3960642 1.4420643]]
Epoch: 1 / 100, Error: 1.8421691681172918
-inputs: [[ 0.37952033  0.18595143 -0.05036932 -0.7908334 ]
 [ 0.24565826 -0.0274782   0.64614284  0.3106448 ]]
np.exp( -inputs ) [[1.4615831  1.2043638  0.9508782  0.4534667 ]
 [1.2784626  0.97289586 1.9081665  1.3643045 ]]
-inputs: [[ 0.4048269  -0.2315818   0.401436    0.03653459]
 [ 0.46884766 -

/tmp/ipykernel_419599/4293650904.py:78: RuntimeWarning: overflow encountered in exp
  print( "np.exp( -inputs )",np.exp( -inputs  ) );
/tmp/ipykernel_419599/4293650904.py:79: RuntimeWarning: overflow encountered in exp
  return 1 / ( 1 + np.exp( -inputs ) );
/tmp/ipykernel_419599/4293650904.py:73: RuntimeWarning: overflow encountered in multiply
  return inputs * ( 1 - inputs );
/home/protim/Documents/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [780]:
print( nn.predict( x_test ) );

-inputs: [[nan nan nan nan]
 [nan nan nan nan]]
np.exp( -inputs ) [[nan nan nan nan]
 [nan nan nan nan]]
-inputs: [[nan nan nan nan]
 [nan nan nan nan]]
np.exp( -inputs ) [[nan nan nan nan]
 [nan nan nan nan]]
[0 0 0 0]
